<a href="https://colab.research.google.com/github/jopelle/Practica3_Ingenieria_Software/blob/master/Probando.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparación del dataset**

Importamos las librerias que van a ser necesarias para realizar esta práctica

In [1]:
!pip install fastai2

     |████████████████████████████████| 194kB 2.7MB/s 


In [0]:
import sys
from fastai.text import *
import html
import json
import numpy as np
from pathlib import Path
from fastai2.text.all import *

Descomprimimos la libreria noticias 

In [0]:
!unzip noticias.zip -d noticias

Mostramos todos los nombres de los ficheros que se encuentran en nuestra carpeta noticias.

Guardamos el contenido de los ficheros en la variable jsons.

In [0]:
path=Path('./noticias')
jsons=[]
for file in path.iterdir():
    print(file)
    for line in open(file, 'r', encoding="utf-8"):
        jsons.append(json.loads(line))

Ahora tenemos el contenido de cada fichero json en una componente de la varialbe jsons.

Mostramos los keys de los json

In [0]:
jsons[0].keys()

Nos interesa el contenido de 'result', ya que los otros no aportan mucho.


In [0]:
jsons[0]['result'].keys()

In [0]:
for k in jsons[0]['result'].keys():
    print(k, type(jsons[0]['result'][k]))

In [0]:
jsons[0]['result']['facets']

In [0]:
jsons[0]['result']['results']

Como podemos ver, en 'results' tenemos una lista con la información de cada noticia.

In [0]:
type(jsons[0]['result']['results'][0])

La descripción de las noticias es la información que nos interesa y las descripciones serán nuestras clases

In [0]:
org='organizacion'
desc='descripcion'

In [0]:
etiquetas = []
descripciones = []
organizaciones = []

for ficheroJson in jsons:
    for noticia in ficheroJson['result']['results']:
        if 'organizacion' in noticia and len(noticia[org])==1 and len(noticia[desc])>0:
            etiquetas.append(noticia[org][0])
            descripciones.append(noticia[desc])
            organizaciones.append(noticia[org][0])

Transformamos las organizaciones en números enteros

In [0]:
def labels_to_num(tags, labels_dict):
    labels = [labels_dict[i] for i in tags]
    labels = np.array(labels)
    return labels

str_labels = set(etiquetas)
labels_dict = {tag: i for i, tag in enumerate(str_labels)}

organizaciones = labels_to_num(train_organizaciones, labels_dict)

Con el siguiente código "limpiamos" las descripciones de elementos html

In [0]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [0]:
def limpiarDescripcion(descs):
  descsBien = []
  for i in range(len(descs)):
      a = cleanhtml(descs[i])
      a = a.replace("\r\n"," ")
      a = html.unescape(a)
      a = a.strip()
      descsBien.append(a)
  return descsBien

In [0]:
descBien = limpiarDescripcion(descripciones)

In [0]:
np.random.seed(42)

trn_idx = np.random.permutation(len(tweets))

tweets = tweets[trn_idx]
labels = labels[trn_idx]
types = types[trn_idx]